# Exploring Network History in NeKo

This notebook demonstrates the new history facilities in NeKo. You will learn how to:

- capture network states automatically while you enrich a network
- inspect branches and metadata associated with each state
- visualise the history graph as both a `networkx` object and an embedded SVG
- control the history recorder with context managers and global toggles
- prune older history entries using `set_max_history`

In [1]:
import pandas as pd
from IPython.display import display, HTML
from neko.core.network import Network

## 1. Create a network using the Omnipath database

In [2]:
net = Network(initial_nodes=["SRC", "CDH1", "TP53"], resources="omnipath")
net.list_states()

[{'id': 0, 'metadata': {'label': 'initial'}}]

## 2. Automatic history capture
Every mutating API call creates a new state with metadata describing the operation.

In [3]:
net.add_node("TP63")
net.add_node("FAK")
net.list_states()

[{'id': 0, 'metadata': {'label': 'initial'}},
 {'id': 1, 'metadata': {'method': 'add_node', 'args': ['TP63'], 'kwargs': {}}},
 {'id': 2, 'metadata': {'method': 'add_node', 'args': ['PTK2'], 'kwargs': {}}}]

We can compare branches or move around the history using `checkout`, `undo`, and `redo`.

In [4]:
current = net.current_state_id
print("Current state ID:", current)

Current state ID: 2


In [5]:
net.undo()

In [6]:
net.nodes

,Genesymbol,Uniprot,Type
0,SRC,P12931,NaN
1,CDH1,P12830,NaN
2,TP53,P04637,NaN
3,TP63,Q9H3D4,NaN


In [7]:
net.redo()
net.nodes

,Genesymbol,Uniprot,Type
0,SRC,P12931,NaN
1,CDH1,P12830,NaN
2,TP53,P04637,NaN
3,TP63,Q9H3D4,NaN
4,PTK2,Q05397,NaN


## 3. Visualising the history graph
`history_graph()` returns a `networkx` object, while `history_html()` embeds a Graphviz SVG.

In [8]:
history_nx = net.history_graph()
list(history_nx.edges())

[(0, 1), (1, 2)]

In [9]:
HTML(net.history_html())

## 4. Working with branches
We can revert to an earlier state, explore an alternative enrichment strategy, and compare the resulting states.

In [25]:
root_state = net.root_state_id
net.checkout(root_state)
net.add_node("P01112")
branch_state = net.current_state_id
diff = net.compare_states(current, branch_state)
diff

{'added_nodes': ['P01112'],
 'removed_nodes': ['Q05397', 'Q9H3D4'],
 'added_edges': [],
 'removed_edges': [],
 'added_provenance': [],
 'removed_provenance': []}

## 5. Suspension and toggles
Use `suspend_history()` or `set_history_tracking(False)` to batch changes without recording intermediate states.

In [26]:
with net.suspend_history():
    net.add_node("Q9Y243")  # already present, but no state recorded
    net.list_states()[-1]


In [27]:
net.set_history_tracking(False)
net.add_node("Q15796")
net.set_history_tracking(True)
net.list_states()


[{'id': 0, 'metadata': {'label': 'initial'}},
 {'id': 1,
  'metadata': {'method': 'add_node', 'args': ["'TP63'"], 'kwargs': {}}},
 {'id': 2,
  'metadata': {'method': 'add_node', 'args': ["'FAK'"], 'kwargs': {}}},
 {'id': 3,
  'metadata': {'method': 'add_node', 'args': ["'P01112'"], 'kwargs': {}}}]

## 6. Pruning history
`set_max_history()` limits the number of stored states while preserving the root and the latest state.

In [28]:
net.set_max_history(4)
for node in ["Q9Y6K9", "P17302"]:
    if node not in net.nodes["Genesymbol"].tolist():
        net.add_node(node)
net.list_states()


[{'id': 0, 'metadata': {'label': 'initial'}},
 {'id': 3,
  'metadata': {'method': 'add_node', 'args': ["'P01112'"], 'kwargs': {}}},
 {'id': 4,
  'metadata': {'method': 'add_node', 'args': ["'Q9Y6K9'"], 'kwargs': {}}},
 {'id': 5,
  'metadata': {'method': 'add_node', 'args': ["'P17302'"], 'kwargs': {}}}]

When the number of stored states exceeds the limit, older leaf states are pruned automatically. The root and current state remain intact.

This concludes the walkthrough of the history API. Experiment by combining these features with your favourite enrichment strategies to analyse multiple hypotheses side by side.

In [10]:
net = Network(initial_nodes=["SRC", "CDH1", "TP53"], resources="omnipath")

In [11]:
net.complete_connection(maxlen=2, algorithm="bfs", only_signed=True, consensus=True)

In [12]:
net.edges

,source,target,Type,Effect,References,Provenance
0,P12830,P12931,None,stimulation,ACSN:16039586;ACSN:16099633;ACSN:17143292;ACSN...,None
1,P12931,P07949,None,stimulation,BioGRID:10070972;HPRD:10070972;KEA:11713247;KE...,None
2,P07949,P12830,None,stimulation,NaN,None
3,P04637,P14210,None,stimulation,SPIKE:18431400;SPIKE_LC:18431400; SPIKE:184314...,None
4,P14210,P12931,None,stimulation,ProtMapper:23524339; ProtMapper:23524339,None
5,P12931,Q14289,None,stimulation,Adhesome:10329689;Adhesome:10521452;Adhesome:1...,None
6,Q14289,P04637,None,stimulation,NaN,None
7,P04637,Q15910,None,inhibition,NaN,None
8,Q15910,P12830,None,inhibition,ACSN:11309385;ACSN:12508111;ACSN:14673164;ACSN...,None
9,P12830,P63000,None,stimulation,NaN,None


In [13]:
net.undo()

In [14]:
net.edges

,source,target,Type,Effect,References,Provenance


In [15]:
net.connect_network_radially(max_len=1, only_signed=True, consensus=True)

In [16]:
net.edges

,source,target,Type,Effect,References,Provenance
13,P12830,P12931,None,stimulation,ACSN:16039586;ACSN:16099633;ACSN:17143292;ACSN...,None
14,P12830,P01133,None,inhibition,NaN,None
15,P12830,P62834,None,stimulation,NaN,None
20,P12830,P14210,None,inhibition,NaN,None
23,P12830,P63000,None,stimulation,NaN,None
...,...,...,...,...,...,...
915,P08151,P04637,None,inhibition,NaN,None
932,P61978,P04637,None,stimulation,InnateDB:23092970;SPIKE:16360036;SPIKE:1847200...,None
957,P98170,P04637,None,stimulation,NaN,None
962,P09038,P04637,None,inhibition,NaN,None


In [17]:
net.list_states()

[{'id': 0, 'metadata': {'label': 'initial'}},
 {'id': 1,
  'metadata': {'method': 'complete_connection',
   'args': [],
   'kwargs': {'maxlen': '2',
    'algorithm': 'bfs',
    'only_signed': 'True',
    'consensus': 'True'}}},
 {'id': 2,
  'metadata': {'method': 'connect_network_radially',
   'args': [],
   'kwargs': {'max_len': '1', 'only_signed': 'True', 'consensus': 'True'}}}]

In [18]:
history_nx = net.history_graph()
list(history_nx.edges())

[(0, 1), (0, 2)]

In [33]:
HTML(net.history_html())

In [20]:
net.compare_states(1, 2)

{'added_nodes': ['GJA1',
  'HIPK2',
  'SNAI2',
  'ITGB6',
  'ITGB8',
  'PTP4A1',
  'PRKD1',
  'ZYX',
  'MAPK11',
  'EPHB1',
  'PIK3R3',
  'HMGB1',
  'EGF',
  'RAP1A',
  'FGF2',
  'PPP2CA',
  'DEK',
  'HSPA1B',
  'NFKB1',
  'AR',
  'MAPK1',
  'CDC42',
  'GDNF',
  'FOS',
  'HNRNPK',
  'TLN1',
  'PDGFD',
  'NUMB',
  'GRK2',
  'BCL2L1',
  'WRN',
  'PDGFRB',
  'CTNND1',
  'MAPK14',
  'GFRA1',
  'PAK4',
  'GLI1',
  'BCL6',
  'CTNNA1',
  'ITGB5',
  'DAPK1',
  'ITGB4',
  'MET',
  'NR3C1',
  'VAV1',
  'XIAP',
  'MAPK3',
  'PRKCD',
  'HSPA1A',
  'S100A4'],
 'removed_nodes': [],
 'added_edges': [('MAPK11',
   'TP53',
   None,
   'stimulation',
   'ACSN:11274345;ACSN:15140942;ACSN:15187187;ACSN:16393692;ACSN:19436832;ACSN:20506250;ACSN:21614932;ProtMapper:10212189;SIGNOR:17254968;SPIKE_LC:17145710',
   None),
  ('TP53', 'DEK', None, 'inhibition', nan, None),
  ('MAPK1',
   'TP53',
   None,
   'stimulation',
   'BioGRID:15867353;HPRD-phos:11409876;HPRD-phos:15116093;HPRD:11409876;HPRD:12091386;HPRD

In [21]:
net.checkout(1)

In [27]:
net.convert_edgelist_into_genesymbol()

,source,target,Type,Effect,References,Provenance
0,CDH1,SRC,None,stimulation,ACSN:16039586;ACSN:16099633;ACSN:17143292;ACSN...,None
1,SRC,RET,None,stimulation,BioGRID:10070972;HPRD:10070972;KEA:11713247;KE...,None
2,RET,CDH1,None,stimulation,NaN,None
3,TP53,HGF,None,stimulation,SPIKE:18431400;SPIKE_LC:18431400; SPIKE:184314...,None
4,HGF,SRC,None,stimulation,ProtMapper:23524339; ProtMapper:23524339,None
5,SRC,PTK2B,None,stimulation,Adhesome:10329689;Adhesome:10521452;Adhesome:1...,None
6,PTK2B,TP53,None,stimulation,NaN,None
7,TP53,EZH2,None,inhibition,NaN,None
8,EZH2,CDH1,None,inhibition,ACSN:11309385;ACSN:12508111;ACSN:14673164;ACSN...,None
9,CDH1,RAC1,None,stimulation,NaN,None


In [29]:
net.add_node("MMP14")
net.add_node("CASP3")

True

In [31]:
net.connect_component(net.nodes["Genesymbol"].tolist(), ["MMP14", "CASP3"], maxlen=2, only_signed=True, consensus=True)

In [32]:
HTML(net.history_html())

In [ ]:
net.history_html().